In [9]:
# %load '/home/1529293_snhu/Documents/animalshelter.py'
#     Program: Austin Animal Shelter CRUD
#      Author: Jeremy Morrison
#        Date: 12/12/21
# Description: Module connects to MongoDB and allows users to utilize CRUD functionality

from pymongo import MongoClient
from bson.objectid import ObjectId

class AnimalShelter(object):
    """ CRUD operations for Animal collection in MongoDB """

    def __init__(self, username, password):
        # Initializing the MongoClient. This helps to 
        # access the MongoDB databases and collections. 
        self.client = MongoClient('mongodb://%s:%s@localhost:44409/AAC' % (username, password))
        self.database = self.client['AAC']
      

# Complete this create method to implement the C in CRUD.
    def create(self, data):
        if data is not None:
            self.database.animals.insert(data)  # data should be dictionary
            print("True")
        else:
            raise Exception("False")

# Create method to implement the R in CRUD. 
    def read(self, data):
        if data is not None:
            # Leaving out ObjectID for Pandas DataFrame
            return self.database.animals.find(data, { '_id': 0} )
                
        else:
             raise Exception("Parameters returned no records")
                
# Update item based on parameter, which is the U in CRUD.
    def update(self, query, data):
        if data is not None:
            self.database.animals.update_many(query, data)
            print("Updated")
            for x in self.database.animals.find(query):
                print(x)
        else:
            raise Exception("Couldn't update record")
            
# Delete record based on parameter, which is the D in CRUD.
    def delete(self, data):
        if data is not None:
            self.database.animals.delete_many(data)
            for x in self.database.animals.find(data):
                print(x)
        else:
            raise Exception("Couldn't find record to delete")

In [10]:
# Program: Project 2 Dashboard
# Author: Jeremy Morrison
# Date: 12/12/21
# Description: Program uilizes animalshelter module to read from The Austin Animal Center Outcomes data set
#              to create a Data Table with filter buttons. The data is also used to create a histogram and
#              map of the various animals available for Grazioso Salvare

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import sys
# Appending filepath to sys.path
sys.path.append("/home/1529293_snhu/Documents/")
# Importing AnimalShelter class from my module aac_crud
from animalshelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Jeremy Morrison Project 2 Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Allows 10 rows per page, and starts on page 0. Columns and Rows may be selected but not deleted/edited
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
#             figure = px.pie(dff, values='breed',  
#                             title='Available Dogs by Breed')
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


app

ServerSelectionTimeoutError: localhost:44409: [Errno 111] Connection refused